In [4]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Literal, Annotated, operator

load_dotenv()


True

In [5]:
from langchain_groq import ChatGroq
from langchain_deepseek import ChatDeepSeek
from langgraph.graph import StateGraph, START, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import OutputFixingParser


In [6]:
LLMS = {}

LLMS['gemini'] = ChatGoogleGenerativeAI(
    api_key = os.getenv("GOOGLE_API_KEY"),
    model = 'gemini-1.5-flash'
)

LLMS['deepseek'] = ChatDeepSeek(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    api_base=os.getenv("OPENROUTER_BASE_URL"),
    model="deepseek/deepseek-chat-v3-0324:free"
)

LLMS['moonshotai'] = ChatGroq(
    api_key = os.getenv("GROQ_API_KEY"),
    model = "moonshotai/kimi-k2-instruct"
)


In [7]:
generator_llm: ChatGoogleGenerativeAI = LLMS['gemini']
evaluator_llm: ChatGroq = LLMS['moonshotai']
optimizer_llm: ChatDeepSeek = LLMS['deepseek']
parser = OutputFixingParser.from_llm(llm = LLMS['moonshotai'], parser=JsonOutputParser())


In [20]:
class TweetState(TypedDict):
    topic: str
    tweet: str
    approved: Literal["yes", "no"]
    feedback: str
    iteration: int
    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]
    max_iteration: int


In [21]:
def generate_tweet(state: TweetState):
    topic = state['topic'];
    generate_tweet_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are an expert in writing funny tweets or Twitter posts.

        # Instructions
        You write tweets in a funny way that makes people happy.
        Your humor can be observational, sarcastic, witty, or absurd—whatever fits the topic best.
        Tweets should be short, punchy, and feel like they belong on Twitter.
        Use emojis when they enhance the joke or add personality, but don't overdo it.
        Avoid offensive or divisive content. Keep it light, clever, and relatable.
        Each tweet should be self-contained and not rely on hashtags or images.
        Stick to a character limit of 280 or less.

        # Output Format
        Your output must be in **JSON** format only.
        The output format is {{'tweet': <your_written_tweet>}}

        # Example
        Input - Write a tweet on people who check their fridge multiple times hoping new food appears.  
        Output - {{ 'tweet': "Me: *opens fridge for the 5th time in 10 minutes*\nAlso me: Maybe the food fairy came this time" }}

        Input - Write a tweet about how hard waking up is.  
        Output - {{'tweet': "Alarm: *goes off*\\nMe: who the hell scheduled this meeting with life?? 😩⏰"}}
        """),
        ("human", "Write a tweet on {topic}")
    ])
    generator_chain = generate_tweet_prompt | generator_llm | parser
    result = generator_chain.invoke(input={
        'topic': topic
    })
    state['tweet'] = result['tweet']
    
    return state


In [22]:
def evaluate_tweet(state: TweetState):
    topic = state['topic']
    tweet = state['tweet']
    evaluate_tweet_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are an expert in evaluating funny Twitter posts or tweets.

        # Instructions
        Your job is to evaluate whether a tweet is funny, well-written, and fits the tone of modern Twitter humor.
        Good tweets are punchy, clever, relatable, or absurd in a way that makes people laugh or nod in recognition.
        You should also explain *why* a tweet did or didn't work, so the writer can improve it on the next try.

        ### Evaluation Criteria
        - Is the tweet actually funny or clever?
        - Does it have strong timing, a twist, or surprise?
        - Is the idea relatable or emotionally resonant?
        - Does it avoid clichés or overused formats?
        - Is the language natural, clear, and tweet-worthy?
        - Are emojis used appropriately (if at all)?
        - Is the tweet under 280 characters?

        ### Output Rules
        - Your response must be in **JSON** format only.
        - Always respond with **both** an 'approved' field and a 'feedback' field.
        - 'approved' must be either **"yes"** or **"no"** (no other values).
        - 'feedback' should be a short explanation that helps improve the tweet. Keep it constructive and clear. If the tweet is great, say what worked.

        # Output Format
        {{
        'approved': '<yes or no>',
        'feedback': '<helpful feedback to guide improvement>'
        }}

        # Examples
        Input - "Me: *opens fridge for the 5th time in 10 minutes*\\nAlso me: Maybe the food fairy came this time 🧚‍♂️"  
        Output - {{
        'approved': 'yes',
        'feedback': 'Relatable setup with a whimsical twist. Punchline works well, and the emoji adds tone without overdoing it.'
        }}

        Input - "I made a sandwich today. That's it. That's the tweet."  
        Output - {{
        'approved': 'no',
        'feedback': 'This format is tired and overused. The idea is too plain without a twist or punchline. Needs more personality or exaggeration.'
        }}

        Input - "Shoutout to my alarm clock for being the most consistent relationship I've ever had ❤️⏰"  
        Output - {{
        'approved': 'no',
        'feedback': 'Clichéd and overdone. This type of “alarm clock as partner” joke has been tweeted thousands of times. Try a fresher angle.'
        }}

        Input - "Sometimes I read my old texts and wonder how I've made it this far without being arrested"  
        Output - {{
        'approved': 'yes',
        'feedback': 'Funny, chaotic energy with a dark twist. It surprises the reader and feels personal without being off-putting.'
        }}

        Input - "Just asked my plants if they're proud of me. No response, but I felt seen 🌿"  
        Output - {{
        'approved': 'yes',
        'feedback': 'Light, silly, and emotionally weird in a good way. The tone matches modern Twitter humor perfectly.'
        }}

        Input - "When life gives you lemons, throw them at your enemies and run"  
        Output - {{
        'approved': 'no',
        'feedback': 'This structure is way overused. The twist is slightly funny, but the overall format feels like a tired meme template.'
        }}

        Input - "I miss pizza.\\nAnd by pizza I mean happiness."  
        Output - {{
        'approved': 'yes',
        'feedback': 'This one's dry, short, and lands well. The turn in the second line gives it punch and a bit of sadness—Twitter gold.'
        }}
        """),
        ("human", """
         Evaluate this tweet: {tweet}.
         Here is the topic on which, tweet has been generated: {topic}
         """)
    ])
    evaluator_chain = evaluate_tweet_prompt | evaluator_llm | parser
    result = evaluator_chain.invoke(input={
        'tweet':tweet,
        'topic': topic
    })
    state['approved'] = result['approved']
    state['feedback'] = result['feedback']
    state['tweet_history'] = [tweet]
    state['feedback_history'] = [result['feedback']]
    state['iteration'] = state['iteration'] + 1
    
    return state


In [23]:
def optimize_tweet(state: TweetState):
    topic, tweet, feedback = state['topic'], state['tweet'], state['feedback']
    optimize_tweet_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are a tweet optimizer.

        # Context
        You take a topic, a previously written tweet, and feedback from an evaluator.
        Your goal is to write a better, funnier tweet—based on the same topic—that directly addresses the evaluator's feedback.
        You do NOT explain anything. You just give a new, improved tweet.
        The output should be punchy, clever, and under 280 characters.
        You can use emojis if they fit naturally.
        Avoid clichés, lazy setups, or overused Twitter formats.

        # Instructions
        - Stick to the topic, but you don't need to reuse the previous wording.
        - Use the feedback as your guide. If the tweet lacked a twist, add one. If it was too generic, find a specific or weird take.
        - You can be sarcastic, dry, absurd, or observational—choose whatever humor fits the moment best.
        - Write like a human who understands timing and voice on Twitter.
        - No hashtags, no explanations, no "that's the tweet" endings.
        - Character count must stay under 280.
        - Only respond in the required JSON format.

        # Output Format
        Return only a JSON object like this:
        {{'tweet': '<your new tweet>'}}

        # Examples

        Input:
        Topic - healthy eating  
        Previous Tweet - "Eating salad is like eating disappointment"  
        Feedback - "The joke feels flat and too obvious. It lacks a twist or clever turn. Try exaggerating or adding a more unexpected punchline."  
        Output - {{'tweet': "Salad isn't food. It's what food eats."}}

        Input:
        Topic - working from home  
        Previous Tweet - "Zoom fatigue is real."  
        Feedback - "Too vague. This reads more like a headline than a joke. Add a personal angle or exaggeration."  
        Output - {{'tweet': "Me pretending to care on Zoom while wearing pajama pants and eating cereal out of a mug 🍽️💻"}}

        Input:
        Topic - Mondays  
        Previous Tweet - "Can't believe it's already Monday again"  
        Feedback - "Too generic and overused. It states a common feeling but doesn't add a funny or fresh take. Look for a unique angle."  
        Output - {{'tweet': "Pretty bold of Monday to show up every week uninvited."}}
        """),
        ("human", """Here is the previous tweet with its feedback and topic:
          Topic - {topic}
          Tweet - {tweet}
          Feedback - {feedback}
        """)
    ])
    optimizer_chain = optimize_tweet_prompt | optimizer_llm | parser
    result = optimizer_chain.invoke(input={
        'topic': topic,
        'tweet': tweet,
        'feedback': feedback
    })
    state['tweet'] = result['tweet']
    
    return state


In [24]:
def check_approval(state: TweetState):
    if state["iteration"] > state["max_iteration"]:
        return END
    elif state["iteration"] <= state["max_iteration"] and state["approved"] == 'yes':
        return END
    else:
        return "optimize_tweet"


In [25]:
graph = StateGraph(TweetState);

graph.add_node("generate_tweet", generate_tweet)
graph.add_node("evaluate_tweet", evaluate_tweet)
graph.add_node("optimize_tweet", optimize_tweet)

graph.add_edge(START, "generate_tweet")
graph.add_edge("generate_tweet", "evaluate_tweet")
graph.add_conditional_edges("evaluate_tweet", check_approval)
graph.add_edge("optimize_tweet", "evaluate_tweet")

workflow = graph.compile()


In [28]:
initial_state = {
    'topic': "Charging your phone at 1%",
    'iteration': 0,
    'max_iteration': 3
}

final_state = workflow.invoke(initial_state);


In [29]:
final_state


{'topic': 'Charging your phone at 1%',
 'tweet': 'My phone at 1% is like a dramatic actor taking their final bow—only to reappear for an unscheduled encore the second I find a charger. 👏🔋',
 'approved': 'yes',
 'feedback': 'The metaphor is fresh and theatrical, and the encore twist gives it a punchy payoff. Clean language, under 280 chars, and the emoji choices reinforce the joke without cluttering. Strong execution.',
 'iteration': 2,
 'tweet_history': ["My phone at 1%:  It's not goodbye, it's see you in five minutes... while I frantically search for the charger. 😅🔌",
  "My phone at 1%:  It's not goodbye, it's see you in five minutes... while I frantically search for the charger. 😅🔌",
  'My phone at 1% is like a dramatic actor taking their final bow—only to reappear for an unscheduled encore the second I find a charger. 👏🔋'],
 'feedback_history': ['The ‘phone at 1% pretending to leave’ bit is a meme that’s been done to death. The joke telegraphs itself, and the “see you in five minute